In [1]:
# ============================================================
# IMPORTACION DE LIBRERIAS
# ============================================================

# Instalamos las dependencias necesarias
!pip install networkx matplotlib ipywidgets -q

# Librerias para manejo de datos y estructuras
import numpy as np
from collections import deque
from typing import List, Tuple, Set, Optional
import time
import random
from abc import ABC, abstractmethod

# Librerias para visualizacion
import networkx as nx
import matplotlib.pyplot as plt

# Librerias para interfaz grafica en Colab
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from google.colab import files

print("Sistema inicializado correctamente")
print("Todas las librerias han sido cargadas")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.4 MB/s eta 0:00:00
Sistema inicializado correctamente
Todas las librerias han sido cargadas


In [2]:
# ============================================================
# CLASE: MatrizDispersaCSR
# ============================================================
# Esta clase implementa el formato Compressed Sparse Row para
# almacenar matrices de adyacencia de forma eficiente.
# En lugar de guardar una matriz completa NxN llena de ceros,
# solo almacenamos las conexiones que realmente existen.
# ============================================================

class MatrizDispersaCSR:
    """
    Representa una matriz dispersa usando el formato CSR.

    El formato CSR utiliza tres arreglos:
    - valores: contiene los pesos de las aristas (1 para grafos no ponderados)
    - indices_columnas: almacena el indice del nodo destino de cada arista
    - punteros_fila: indica donde comienza cada fila en el arreglo de valores

    Esto reduce drasticamente el uso de memoria en grafos dispersos.
    """

    def __init__(self, numero_nodos: int):
        """
        Inicializa la estructura CSR vacia.

        Parametros:
            numero_nodos: cantidad total de nodos en el grafo
        """
        self.numero_nodos = numero_nodos

        # Arreglo de valores de las aristas
        self.valores = []

        # Arreglo con los indices de las columnas (nodos destino)
        self.indices_columnas = []

        # Arreglo de punteros que marca el inicio de cada fila
        self.punteros_fila = [0]

        print(f"[Sistema] Matriz CSR inicializada para {numero_nodos} nodos")

    def agregar_fila_nodo(self, lista_vecinos: List[int]):
        """
        Agrega una fila completa a la matriz CSR.
        Cada fila representa las conexiones de un nodo especifico.

        Parametros:
            lista_vecinos: lista de nodos a los que este nodo se conecta
        """
        # Por cada vecino, agregamos una entrada en la estructura
        for nodo_vecino in lista_vecinos:
            self.valores.append(1)  # Peso 1 para grafos no ponderados
            self.indices_columnas.append(nodo_vecino)

        # Actualizamos el puntero para indicar donde termina esta fila
        self.punteros_fila.append(len(self.valores))

    def obtener_vecinos(self, nodo: int) -> List[int]:
        """
        Retorna la lista de vecinos de un nodo especifico.

        Parametros:
            nodo: identificador del nodo a consultar

        Retorna:
            Lista de identificadores de nodos vecinos
        """
        # Buscamos donde inicia y termina la informacion de este nodo
        indice_inicio = self.punteros_fila[nodo]
        indice_fin = self.punteros_fila[nodo + 1]

        # Extraemos la sublista correspondiente
        return self.indices_columnas[indice_inicio:indice_fin]

    def calcular_grado(self, nodo: int) -> int:
        """
        Calcula el grado de salida de un nodo.
        El grado es la cantidad de aristas que salen del nodo.

        Parametros:
            nodo: identificador del nodo

        Retorna:
            Numero de conexiones salientes
        """
        return self.punteros_fila[nodo + 1] - self.punteros_fila[nodo]

    def calcular_memoria_usada(self) -> float:
        """
        Estima la cantidad de memoria RAM utilizada por esta estructura.

        Retorna:
            Memoria en megabytes
        """
        # Calculamos bytes totales asumiendo enteros de 4 bytes
        bytes_valores = len(self.valores) * 4
        bytes_indices = len(self.indices_columnas) * 4
        bytes_punteros = len(self.punteros_fila) * 4

        total_bytes = bytes_valores + bytes_indices + bytes_punteros

        # Convertimos a megabytes
        return total_bytes / (1024 * 1024)

print("Clase MatrizDispersaCSR definida correctamente")

Clase MatrizDispersaCSR definida correctamente


In [3]:
# ============================================================
# CLASE ABSTRACTA: GrafoBase
# ============================================================
# Define la interfaz que todo tipo de grafo debe implementar.
# Esto permite tener diferentes implementaciones (disperso,
# denso, etc.) que cumplan con el mismo contrato.
# ============================================================

class GrafoBase(ABC):
    """
    Clase abstracta que define la interfaz para grafos.

    Cualquier implementacion concreta debe proveer estos metodos.
    Esto es un ejemplo de polimorfismo en programacion orientada a objetos.
    """

    @abstractmethod
    def cargar_dataset(self, ruta_archivo: str):
        """
        Carga un grafo desde un archivo en formato Edge List.

        Parametros:
            ruta_archivo: ubicacion del archivo de datos
        """
        pass

    @abstractmethod
    def obtener_vecinos(self, nodo: int) -> List[int]:
        """
        Obtiene los nodos adyacentes a un nodo dado.

        Parametros:
            nodo: identificador del nodo

        Retorna:
            Lista de vecinos
        """
        pass

    @abstractmethod
    def busqueda_amplitud(self, nodo_inicial: int, profundidad_maxima: int) -> Tuple[List[int], List[Tuple[int, int]]]:
        """
        Ejecuta algoritmo BFS desde un nodo.

        Parametros:
            nodo_inicial: nodo desde donde comenzar
            profundidad_maxima: cuantos niveles explorar

        Retorna:
            Tupla con (nodos_visitados, aristas_exploradas)
        """
        pass

    @abstractmethod
    def busqueda_profundidad(self, nodo_inicial: int) -> List[int]:
        """
        Ejecuta algoritmo DFS desde un nodo.

        Parametros:
            nodo_inicial: nodo desde donde comenzar

        Retorna:
            Lista de nodos visitados en orden DFS
        """
        pass

    @abstractmethod
    def encontrar_nodo_critico(self) -> Tuple[int, int]:
        """
        Encuentra el nodo con mayor grado de salida.

        Retorna:
            Tupla (id_nodo, grado)
        """
        pass

    @abstractmethod
    def calcular_distancia(self, nodo_origen: int, nodo_destino: int) -> Optional[int]:
        """
        Calcula la distancia minima entre dos nodos.

        Parametros:
            nodo_origen: nodo de partida
            nodo_destino: nodo de llegada

        Retorna:
            Distancia (numero de saltos) o None si no hay camino
        """
        pass

print("Clase abstracta GrafoBase definida correctamente")

Clase abstracta GrafoBase definida correctamente


In [4]:
# ============================================================
# CLASE CONCRETA: GrafoDisperso
# ============================================================
# Implementacion de un grafo usando estructura CSR.
# Todos los algoritmos estan implementados manualmente sin
# usar librerias de grafos para los calculos.
# ============================================================

class GrafoDisperso(GrafoBase):
    """
    Implementacion concreta de un grafo disperso.

    Utiliza la estructura CSR para almacenamiento eficiente y
    proporciona implementaciones propias de los algoritmos de
    recorrido y analisis.
    """

    def __init__(self):
        """
        Inicializa un grafo disperso vacio.
        """
        self.matriz_csr = None
        self.numero_nodos = 0
        self.numero_aristas = 0
        self.es_dirigido = True

        print("[Nucleo] GrafoDisperso inicializado")

    def cargar_dataset(self, ruta_archivo: str):
        """
        Carga un grafo desde un archivo en formato Edge List.

        El formato esperado es:
        nodo_origen nodo_destino

        Las lineas que comienzan con # son ignoradas como comentarios.
        """
        print(f"[Nucleo] Iniciando carga de dataset: {ruta_archivo}")
        tiempo_inicio = time.time()

        # Primera pasada: leer todas las aristas y encontrar el maximo nodo
        lista_aristas = []
        nodo_maximo = 0

        try:
            with open(ruta_archivo, 'r') as archivo:
                for linea in archivo:
                    # Ignorar lineas de comentario
                    if linea.startswith('#'):
                        continue

                    # Separar la linea en componentes
                    partes = linea.strip().split()

                    if len(partes) >= 2:
                        nodo_origen = int(partes[0])
                        nodo_destino = int(partes[1])

                        lista_aristas.append((nodo_origen, nodo_destino))

                        # Actualizamos el nodo maximo
                        nodo_maximo = max(nodo_maximo, nodo_origen, nodo_destino)
        except FileNotFoundError:
            print(f"[Error] No se pudo encontrar el archivo: {ruta_archivo}")
            return

        # El numero de nodos es el maximo mas uno (porque empezamos en 0)
        self.numero_nodos = nodo_maximo + 1
        self.numero_aristas = len(lista_aristas)

        print(f"[Nucleo] Archivo leido. Construyendo estructura CSR...")

        # Segunda pasada: construir lista de adyacencia temporal
        lista_adyacencia_temporal = [[] for _ in range(self.numero_nodos)]

        for origen, destino in lista_aristas:
            lista_adyacencia_temporal[origen].append(destino)

        # Tercera pasada: convertir a formato CSR
        self.matriz_csr = MatrizDispersaCSR(self.numero_nodos)

        for vecinos in lista_adyacencia_temporal:
            # Ordenamos los vecinos para mejorar la eficiencia de busqueda
            vecinos_ordenados = sorted(vecinos)
            self.matriz_csr.agregar_fila_nodo(vecinos_ordenados)

        tiempo_total = time.time() - tiempo_inicio
        memoria_mb = self.matriz_csr.calcular_memoria_usada()

        print(f"[Nucleo] Carga completada exitosamente")
        print(f"[Nucleo] Nodos: {self.numero_nodos:,} | Aristas: {self.numero_aristas:,}")
        print(f"[Nucleo] Memoria CSR: {memoria_mb:.2f} MB")
        print(f"[Nucleo] Tiempo de carga: {tiempo_total:.2f} segundos")
        print()

    def obtener_vecinos(self, nodo: int) -> List[int]:
        """
        Retorna los vecinos de un nodo desde la estructura CSR.
        """
        if nodo >= self.numero_nodos or nodo < 0:
            return []
        return self.matriz_csr.obtener_vecinos(nodo)

    def busqueda_amplitud(self, nodo_inicial: int, profundidad_maxima: int) -> Tuple[List[int], List[Tuple[int, int]]]:
        """
        Implementacion manual del algoritmo BFS (Breadth-First Search).

        Este algoritmo explora el grafo nivel por nivel, visitando primero
        todos los vecinos directos, luego los vecinos de los vecinos, etc.

        Usa una cola FIFO (First In First Out) para mantener el orden de
        exploracion por niveles.
        """
        print(f"[Cython] Solicitud BFS recibida: nodo {nodo_inicial}, profundidad {profundidad_maxima}")
        print(f"[Nucleo] Ejecutando BFS nativo...")

        tiempo_inicio = time.time()

        # Conjunto para rastrear nodos ya visitados
        nodos_visitados = set([nodo_inicial])

        # Lista para almacenar las aristas del subgrafo resultante
        aristas_encontradas = []

        # Cola FIFO: cada elemento es (nodo, nivel_actual)
        cola_exploracion = deque([(nodo_inicial, 0)])

        # Mientras haya nodos por explorar
        while cola_exploracion:
            # Extraemos el primer nodo de la cola
            nodo_actual, nivel_actual = cola_exploracion.popleft()

            # Si ya alcanzamos la profundidad maxima, no exploramos mas
            if nivel_actual >= profundidad_maxima:
                continue

            # Exploramos todos los vecinos del nodo actual
            for nodo_vecino in self.obtener_vecinos(nodo_actual):
                # Registramos la arista
                aristas_encontradas.append((nodo_actual, nodo_vecino))

                # Si el vecino no ha sido visitado, lo agregamos
                if nodo_vecino not in nodos_visitados:
                    nodos_visitados.add(nodo_vecino)
                    cola_exploracion.append((nodo_vecino, nivel_actual + 1))

        tiempo_total_ms = (time.time() - tiempo_inicio) * 1000

        print(f"[Nucleo] BFS completado: {len(nodos_visitados)} nodos alcanzados")
        print(f"[Nucleo] Tiempo de ejecucion: {tiempo_total_ms:.3f} ms")
        print(f"[Cython] Retornando resultados a Python")
        print()

        return list(nodos_visitados), aristas_encontradas

    def busqueda_profundidad(self, nodo_inicial: int) -> List[int]:
        """
        Implementacion manual del algoritmo DFS (Depth-First Search).

        Este algoritmo explora el grafo en profundidad, yendo lo mas lejos
        posible antes de retroceder.

        Usa una pila LIFO (Last In First Out) para mantener el orden de
        exploracion en profundidad.
        """
        print(f"[Nucleo] Ejecutando DFS desde nodo {nodo_inicial}...")

        tiempo_inicio = time.time()

        # Conjunto de nodos visitados
        nodos_visitados = set()

        # Lista para mantener el orden de visita
        orden_visita = []

        # Pila para DFS
        pila_exploracion = [nodo_inicial]

        while pila_exploracion:
            # Sacamos el ultimo nodo agregado (LIFO)
            nodo_actual = pila_exploracion.pop()

            # Si ya fue visitado, continuamos
            if nodo_actual in nodos_visitados:
                continue

            # Marcamos como visitado
            nodos_visitados.add(nodo_actual)
            orden_visita.append(nodo_actual)

            # Agregamos todos los vecinos a la pila
            # Los agregamos en orden inverso para mantener el orden natural
            vecinos = self.obtener_vecinos(nodo_actual)
            for vecino in reversed(vecinos):
                if vecino not in nodos_visitados:
                    pila_exploracion.append(vecino)

        tiempo_total_ms = (time.time() - tiempo_inicio) * 1000

        print(f"[Nucleo] DFS completado: {len(orden_visita)} nodos alcanzados")
        print(f"[Nucleo] Tiempo de ejecucion: {tiempo_total_ms:.3f} ms")
        print()

        return orden_visita

    def encontrar_nodo_critico(self) -> Tuple[int, int]:
        """
        Encuentra el nodo con el mayor grado de salida.

        El nodo critico es aquel que tiene mas conexiones, lo cual
        lo hace importante para la robustez de la red.
        """
        print("[Nucleo] Calculando nodo critico...")

        grado_maximo = 0
        nodo_critico = 0

        # Recorremos todos los nodos
        for nodo in range(self.numero_nodos):
            grado_actual = self.matriz_csr.calcular_grado(nodo)

            if grado_actual > grado_maximo:
                grado_maximo = grado_actual
                nodo_critico = nodo

        print(f"[Nucleo] Nodo critico encontrado: {nodo_critico}")
        print(f"[Nucleo] Grado de salida: {grado_maximo}")
        print()

        return nodo_critico, grado_maximo

    def calcular_distancia(self, nodo_origen: int, nodo_destino: int) -> Optional[int]:
        """
        Calcula la distancia minima entre dos nodos usando BFS.

        La distancia es el numero minimo de aristas que hay que recorrer
        para ir del nodo origen al nodo destino.
        """
        print(f"[Nucleo] Calculando distancia de {nodo_origen} a {nodo_destino}...")

        # Si son el mismo nodo, la distancia es 0
        if nodo_origen == nodo_destino:
            print(f"[Nucleo] Distancia: 0 (mismo nodo)")
            print()
            return 0

        # Usamos BFS para encontrar el camino mas corto
        visitados = set([nodo_origen])
        cola = deque([(nodo_origen, 0)])  # (nodo, distancia)

        while cola:
            nodo_actual, distancia_actual = cola.popleft()

            # Exploramos los vecinos
            for vecino in self.obtener_vecinos(nodo_actual):
                # Si encontramos el destino, retornamos la distancia
                if vecino == nodo_destino:
                    distancia_final = distancia_actual + 1
                    print(f"[Nucleo] Distancia encontrada: {distancia_final}")
                    print()
                    return distancia_final

                # Si no ha sido visitado, lo agregamos
                if vecino not in visitados:
                    visitados.add(vecino)
                    cola.append((vecino, distancia_actual + 1))

        # Si llegamos aqui, no hay camino
        print(f"[Nucleo] No existe camino entre los nodos")
        print()
        return None

print("Clase GrafoDisperso implementada correctamente")

Clase GrafoDisperso implementada correctamente


In [5]:
# ============================================================
# FUNCIONES DE VISUALIZACION
# ============================================================
# Estas funciones usan NetworkX UNICAMENTE para dibujar.
# Todos los calculos ya fueron realizados por el nucleo.
# ============================================================

def visualizar_subgrafo(lista_nodos: List[int],
                       lista_aristas: List[Tuple[int, int]],
                       nodo_central: Optional[int] = None,
                       titulo: str = "Subgrafo Resultante"):
    """
    Dibuja un subgrafo usando NetworkX como motor de renderizado.

    Parametros:
        lista_nodos: nodos a visualizar
        lista_aristas: aristas a visualizar
        nodo_central: nodo a destacar con color diferente
        titulo: titulo del grafico
    """
    # Creamos un grafo dirigido vacio
    G = nx.DiGraph()

    # Agregamos los nodos y aristas calculados previamente
    G.add_nodes_from(lista_nodos)
    G.add_edges_from(lista_aristas)

    # Configuramos el tamaño de la figura
    plt.figure(figsize=(14, 10))

    # Elegimos el algoritmo de layout segun el tamaño
    if len(lista_nodos) < 100:
        # Para grafos pequeños, usamos spring con mas iteraciones
        posiciones = nx.spring_layout(G, k=0.5, iterations=50)
    else:
        # Para grafos grandes, usamos menos iteraciones para rapidez
        posiciones = nx.spring_layout(G, k=1, iterations=20)

    # Asignamos colores a los nodos
    colores_nodos = []
    for nodo in G.nodes():
        if nodo == nodo_central:
            colores_nodos.append('#d62728')  # Rojo para nodo central
        else:
            colores_nodos.append('#1f77b4')  # Azul para el resto

    # Dibujamos los nodos
    nx.draw_networkx_nodes(G, posiciones,
                          node_color=colores_nodos,
                          node_size=400,
                          alpha=0.9)

    # Dibujamos las aristas
    nx.draw_networkx_edges(G, posiciones,
                          edge_color='#888888',
                          arrows=True,
                          arrowsize=15,
                          alpha=0.5,
                          width=1.5)

    # Agregamos etiquetas solo si hay pocos nodos
    if len(lista_nodos) < 50:
        nx.draw_networkx_labels(G, posiciones,
                               font_size=9,
                               font_weight='bold')

    # Configuramos el titulo y removemos los ejes
    plt.title(f"{titulo}\n{len(lista_nodos)} nodos, {len(lista_aristas)} aristas",
             fontsize=16,
             fontweight='bold',
             pad=20)
    plt.axis('off')
    plt.tight_layout()
    plt.show()

def mostrar_estadisticas(grafo: GrafoDisperso):
    """
    Muestra un resumen estadistico del grafo cargado.

    Parametros:
        grafo: instancia del grafo a analizar
    """
    print("=" * 70)
    print("ESTADISTICAS DEL GRAFO")
    print("=" * 70)
    print(f"Numero de nodos:          {grafo.numero_nodos:,}")
    print(f"Numero de aristas:        {grafo.numero_aristas:,}")
    print(f"Memoria CSR utilizada:    {grafo.matriz_csr.calcular_memoria_usada():.2f} MB")
    print(f"Tipo de grafo:            {'Dirigido' if grafo.es_dirigido else 'No dirigido'}")

    # Calculamos densidad
    max_aristas_posibles = grafo.numero_nodos * (grafo.numero_nodos - 1)
    densidad = (grafo.numero_aristas / max_aristas_posibles) * 100 if max_aristas_posibles > 0 else 0
    print(f"Densidad del grafo:       {densidad:.6f}%")

    print("=" * 70)
    print()

print("Funciones de visualizacion definidas correctamente")

Funciones de visualizacion definidas correctamente


In [6]:
# ============================================================
# GENERADOR DE GRAFOS SINTETICOS
# ============================================================
# Genera grafos con distribucion power-law que simulan
# redes reales como redes sociales o Internet.
# ============================================================

def generar_grafo_sintetico(cantidad_nodos: int,
                           cantidad_aristas: int,
                           archivo_salida: str,
                           semilla: int = 42):
    """
    Genera un grafo aleatorio con distribucion power-law.

    En redes reales, algunos nodos tienen muchas conexiones (hubs)
    mientras que la mayoria tiene pocas. Esta funcion simula ese
    comportamiento.

    Parametros:
        cantidad_nodos: numero de nodos a generar
        cantidad_aristas: numero de aristas a generar
        archivo_salida: nombre del archivo donde guardar
        semilla: semilla para reproducibilidad
    """
    print(f"Generando grafo sintetico...")
    print(f"  Nodos: {cantidad_nodos:,}")
    print(f"  Aristas objetivo: {cantidad_aristas:,}")

    random.seed(semilla)

    # Usamos un conjunto para evitar aristas duplicadas
    conjunto_aristas = set()

    # Generamos aristas hasta alcanzar la cantidad deseada
    while len(conjunto_aristas) < cantidad_aristas:
        # Con 30% de probabilidad, conectamos con un nodo hub
        # Los hubs son el 10% de los nodos con ID mas bajo
        if random.random() < 0.3:
            nodo_origen = random.randint(0, cantidad_nodos // 10)
        else:
            nodo_origen = random.randint(0, cantidad_nodos - 1)

        nodo_destino = random.randint(0, cantidad_nodos - 1)

        # Evitamos auto-conexiones
        if nodo_origen != nodo_destino:
            conjunto_aristas.add((nodo_origen, nodo_destino))

    # Escribimos el archivo en formato Edge List
    with open(archivo_salida, 'w') as archivo:
        archivo.write(f"# Grafo sintetico generado automaticamente\n")
        archivo.write(f"# Nodos: {cantidad_nodos}\n")
        archivo.write(f"# Aristas: {len(conjunto_aristas)}\n")
        archivo.write(f"# Formato: nodo_origen nodo_destino\n")

        for origen, destino in conjunto_aristas:
            archivo.write(f"{origen} {destino}\n")

    print(f"Grafo generado exitosamente")
    print(f"  Aristas generadas: {len(conjunto_aristas):,}")
    print(f"  Archivo: {archivo_salida}")
    print()

print("Generador de grafos sinteticos definido correctamente")

Generador de grafos sinteticos definido correctamente


In [7]:
# ============================================================
# INICIALIZACION DEL SISTEMA
# ============================================================
# Creamos un grafo de prueba y lo cargamos en memoria
# ============================================================

print("\n" + "=" * 70)
print("INICIANDO SISTEMA NEURONET")
print("=" * 70 + "\n")

# Generamos un dataset de prueba
generar_grafo_sintetico(
    cantidad_nodos=5000,
    cantidad_aristas=25000,
    archivo_salida='grafo_prueba.txt'
)

# Creamos la instancia del grafo
grafo_principal = GrafoDisperso()

# Cargamos los datos
grafo_principal.cargar_dataset('grafo_prueba.txt')

# Mostramos estadisticas iniciales
mostrar_estadisticas(grafo_principal)

print("=" * 70)
print("SISTEMA LISTO PARA USAR")
print("=" * 70)
print()


INICIANDO SISTEMA NEURONET

Generando grafo sintetico...
  Nodos: 5,000
  Aristas objetivo: 25,000
Grafo generado exitosamente
  Aristas generadas: 25,000
  Archivo: grafo_prueba.txt

[Nucleo] GrafoDisperso inicializado
[Nucleo] Iniciando carga de dataset: grafo_prueba.txt
[Nucleo] Archivo leido. Construyendo estructura CSR...
[Sistema] Matriz CSR inicializada para 5000 nodos
[Nucleo] Carga completada exitosamente
[Nucleo] Nodos: 5,000 | Aristas: 25,000
[Nucleo] Memoria CSR: 0.21 MB
[Nucleo] Tiempo de carga: 0.03 segundos

ESTADISTICAS DEL GRAFO
Numero de nodos:          5,000
Numero de aristas:        25,000
Memoria CSR utilizada:    0.21 MB
Tipo de grafo:            Dirigido
Densidad del grafo:       0.100020%

SISTEMA LISTO PARA USAR



In [8]:
# ============================================================
# INTERFAZ GRAFICA DE USUARIO
# ============================================================
# Panel de control interactivo para ejecutar operaciones
# sobre el grafo sin escribir codigo.
# ============================================================

# Creamos los widgets de la interfaz

# Boton para ver estadisticas
boton_estadisticas = widgets.Button(
    description='Ver Estadisticas',
    button_style='info',
    tooltip='Muestra informacion del grafo cargado',
    layout=widgets.Layout(width='200px', height='40px')
)

# Boton para encontrar nodo critico
boton_nodo_critico = widgets.Button(
    description='Nodo Critico',
    button_style='warning',
    tooltip='Encuentra el nodo con mas conexiones',
    layout=widgets.Layout(width='200px', height='40px')
)

# Boton para cargar archivo
boton_cargar_archivo = widgets.Button(
    description='Cargar Archivo',
    button_style='primary',
    tooltip='Cargar un archivo de grafo personalizado',
    layout=widgets.Layout(width='200px', height='40px')
)

# Entrada para nodo inicial de BFS
entrada_nodo_bfs = widgets.IntText(
    value=0,
    description='Nodo inicial BFS:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

# Slider para profundidad de BFS
slider_profundidad_bfs = widgets.IntSlider(
    value=2,
    min=1,
    max=5,
    step=1,
    description='Profundidad:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='400px')
)

# Boton para ejecutar BFS
boton_ejecutar_bfs = widgets.Button(
    description='Ejecutar BFS',
    button_style='success',
    tooltip='Ejecuta busqueda en amplitud',
    layout=widgets.Layout(width='200px', height='40px')
)

# Entrada para nodo inicial de DFS
entrada_nodo_dfs = widgets.IntText(
    value=0,
    description='Nodo inicial DFS:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

# Boton para ejecutar DFS
boton_ejecutar_dfs = widgets.Button(
    description='Ejecutar DFS',
    button_style='success',
    tooltip='Ejecuta busqueda en profundidad',
    layout=widgets.Layout(width='200px', height='40px')
)

# Entradas para calcular distancia
entrada_nodo_origen = widgets.IntText(
    value=0,
    description='Nodo origen:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

entrada_nodo_destino = widgets.IntText(
    value=10,
    description='Nodo destino:',
    style={'description_width': '120px'},
    layout=widgets.Layout(width='300px')
)

# Boton para calcular distancia
boton_calcular_distancia = widgets.Button(
    description='Calcular Distancia',
    button_style='success',
    tooltip='Calcula distancia minima entre dos nodos',
    layout=widgets.Layout(width='200px', height='40px')
)

# Area de salida para resultados
area_salida = widgets.Output()

# Definimos las funciones callback para cada boton

def al_presionar_estadisticas(boton):
    """Muestra las estadisticas del grafo."""
    with area_salida:
        clear_output()
        mostrar_estadisticas(grafo_principal)

def al_presionar_nodo_critico(boton):
    """Encuentra y muestra el nodo critico."""
    with area_salida:
        clear_output()
        nodo_id, grado = grafo_principal.encontrar_nodo_critico()
        print(f"\nNodo critico encontrado: {nodo_id}")
        print(f"Numero de conexiones: {grado}")
        print()

def al_presionar_cargar_archivo(boton):
    """Permite cargar un archivo personalizado."""
    with area_salida:
        clear_output()
        print("Por favor, sube un archivo en formato Edge List")
        print("El archivo debe tener el formato: nodo_origen nodo_destino")
        print()

        # Subimos el archivo
        archivos_subidos = files.upload()

        if archivos_subidos:
            nombre_archivo = list(archivos_subidos.keys())[0]
            print(f"\nArchivo recibido: {nombre_archivo}")
            print("Cargando grafo...\n")

            # Cargamos el nuevo grafo
            global grafo_principal
            grafo_principal = GrafoDisperso()
            grafo_principal.cargar_dataset(nombre_archivo)

            mostrar_estadisticas(grafo_principal)

def al_presionar_ejecutar_bfs(boton):
    """Ejecuta BFS y visualiza el resultado."""
    with area_salida:
        clear_output()

        nodo_inicio = entrada_nodo_bfs.value
        profundidad = slider_profundidad_bfs.value

        # Validamos que el nodo exista
        if nodo_inicio < 0 or nodo_inicio >= grafo_principal.numero_nodos:
            print(f"Error: El nodo {nodo_inicio} no existe en el grafo")
            print(f"Rango valido: 0 a {grafo_principal.numero_nodos - 1}")
            return

        # Ejecutamos BFS
        nodos, aristas = grafo_principal.busqueda_amplitud(nodo_inicio, profundidad)

        print(f"BFS completado exitosamente")
        print(f"Nodos alcanzados: {len(nodos)}")
        print(f"Aristas exploradas: {len(aristas)}")
        print()

        # Visualizamos
        visualizar_subgrafo(nodos, aristas, nodo_inicio, "Resultado BFS")

def al_presionar_ejecutar_dfs(boton):
    """Ejecuta DFS y muestra el resultado."""
    with area_salida:
        clear_output()

        nodo_inicio = entrada_nodo_dfs.value

        # Validamos que el nodo exista
        if nodo_inicio < 0 or nodo_inicio >= grafo_principal.numero_nodos:
            print(f"Error: El nodo {nodo_inicio} no existe en el grafo")
            print(f"Rango valido: 0 a {grafo_principal.numero_nodos - 1}")
            return

        # Ejecutamos DFS
        nodos_visitados = grafo_principal.busqueda_profundidad(nodo_inicio)

        print(f"DFS completado exitosamente")
        print(f"Nodos alcanzados: {len(nodos_visitados)}")
        print(f"\nOrden de visita (primeros 50 nodos):")
        print(nodos_visitados[:50])
        print()

def al_presionar_calcular_distancia(boton):
    """Calcula la distancia entre dos nodos."""
    with area_salida:
        clear_output()

        origen = entrada_nodo_origen.value
        destino = entrada_nodo_destino.value

        # Validamos que los nodos existan
        if origen < 0 or origen >= grafo_principal.numero_nodos:
            print(f"Error: El nodo origen {origen} no existe")
            return

        if destino < 0 or destino >= grafo_principal.numero_nodos:
            print(f"Error: El nodo destino {destino} no existe")
            return

        # Calculamos distancia
        distancia = grafo_principal.calcular_distancia(origen, destino)

        if distancia is not None:
            print(f"\nDistancia minima de {origen} a {destino}: {distancia} saltos")
        else:
            print(f"\nNo existe camino entre {origen} y {destino}")
        print()

# Conectamos los eventos a los botones
boton_estadisticas.on_click(al_presionar_estadisticas)
boton_nodo_critico.on_click(al_presionar_nodo_critico)
boton_cargar_archivo.on_click(al_presionar_cargar_archivo)
boton_ejecutar_bfs.on_click(al_presionar_ejecutar_bfs)
boton_ejecutar_dfs.on_click(al_presionar_ejecutar_dfs)
boton_calcular_distancia.on_click(al_presionar_calcular_distancia)

# Organizamos la interfaz en secciones

# Seccion de informacion general
seccion_general = widgets.VBox([
    widgets.HTML("<h3>Informacion General</h3>"),
    widgets.HBox([boton_estadisticas, boton_nodo_critico, boton_cargar_archivo])
])

# Seccion de BFS
seccion_bfs = widgets.VBox([
    widgets.HTML("<h3>Busqueda en Amplitud (BFS)</h3>"),
    entrada_nodo_bfs,
    slider_profundidad_bfs,
    boton_ejecutar_bfs
])

# Seccion de DFS
seccion_dfs = widgets.VBox([
    widgets.HTML("<h3>Busqueda en Profundidad (DFS)</h3>"),
    entrada_nodo_dfs,
    boton_ejecutar_dfs
])

# Seccion de distancia
seccion_distancia = widgets.VBox([
    widgets.HTML("<h3>Calcular Distancia</h3>"),
    entrada_nodo_origen,
    entrada_nodo_destino,
    boton_calcular_distancia
])

# Interfaz completa
interfaz_completa = widgets.VBox([
    widgets.HTML("<h2>NeuroNet - Panel de Control</h2>"),
    widgets.HTML("<hr>"),
    seccion_general,
    widgets.HTML("<hr>"),
    seccion_bfs,
    widgets.HTML("<hr>"),
    seccion_dfs,
    widgets.HTML("<hr>"),
    seccion_distancia,
    widgets.HTML("<hr>"),
    area_salida
])

# Mostramos la interfaz
display(interfaz_completa)

In [9]:
# ============================================================
# PRUEBAS CON GRAFO MAS GRANDE (OPCIONAL)
# ============================================================
# Esta celda genera y carga un grafo de mayor tamaño para
# demostrar la escalabilidad del sistema.
# ============================================================

print("\n" + "=" * 70)
print("PRUEBA CON GRAFO DE 50,000 NODOS")
print("=" * 70 + "\n")

# Generamos un grafo mas grande
generar_grafo_sintetico(
    cantidad_nodos=50000,
    cantidad_aristas=200000,
    archivo_salida='grafo_grande.txt'
)

# Creamos nueva instancia
grafo_grande = GrafoDisperso()

# Cargamos el grafo
grafo_grande.cargar_dataset('grafo_grande.txt')

# Mostramos estadisticas
mostrar_estadisticas(grafo_grande)

# Encontramos el nodo critico
nodo_critico, grado_critico = grafo_grande.encontrar_nodo_critico()

print(f"Nodo mas conectado: {nodo_critico}")
print(f"Grado: {grado_critico}")
print()

# Probamos BFS
print("Ejecutando BFS en grafo grande...")
nodos_bfs, aristas_bfs = grafo_grande.busqueda_amplitud(nodo_critico, 2)

print(f"Resultado: {len(nodos_bfs)} nodos alcanzados")
print()

print("=" * 70)
print("PRUEBA COMPLETADA")
print("=" * 70)


PRUEBA CON GRAFO DE 50,000 NODOS

Generando grafo sintetico...
  Nodos: 50,000
  Aristas objetivo: 200,000
Grafo generado exitosamente
  Aristas generadas: 200,000
  Archivo: grafo_grande.txt

[Nucleo] GrafoDisperso inicializado
[Nucleo] Iniciando carga de dataset: grafo_grande.txt
[Nucleo] Archivo leido. Construyendo estructura CSR...
[Sistema] Matriz CSR inicializada para 50000 nodos
[Nucleo] Carga completada exitosamente
[Nucleo] Nodos: 50,000 | Aristas: 200,000
[Nucleo] Memoria CSR: 1.72 MB
[Nucleo] Tiempo de carga: 0.40 segundos

ESTADISTICAS DEL GRAFO
Numero de nodos:          50,000
Numero de aristas:        200,000
Memoria CSR utilizada:    1.72 MB
Tipo de grafo:            Dirigido
Densidad del grafo:       0.008000%

[Nucleo] Calculando nodo critico...
[Nucleo] Nodo critico encontrado: 23
[Nucleo] Grado de salida: 29

Nodo mas conectado: 23
Grado: 29

Ejecutando BFS en grafo grande...
[Cython] Solicitud BFS recibida: nodo 23, profundidad 2
[Nucleo] Ejecutando BFS nativo...
[